In [50]:
#Library
import requests
import json
from datetime import datetime
import scipy.optimize as optimize
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('ignore')

In [51]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [52]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,
                      data=json.dumps(content),
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

In [53]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,
                      data=json.dumps(content),
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [54]:
def post_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    #json.dumps(content),
    res = requests.post(url=URL + endpointApi,
                      data=content,
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

Conversion chaque valeur en le type approprié :

In [55]:
def convert_currency(value, df_currency_rate):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value = convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

Création de la table des taux en EUR : 
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [56]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
"""df_currency_rate = getCurrencyRate()
df_currency_rate"""

'df_currency_rate = getCurrencyRate()\ndf_currency_rate'

[TEST] Taux de convertion entre USD et EUR :

In [57]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]

#currencyRateToEUR('USD', df_currency_rate)

[TEST] La database complète de tous les asset :

In [58]:
"""data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets"""

"data = get_data('asset/')\ndf = pd.read_json(data)\ndf_assets = df_convert_type(df, df_currency_rate)\ndf_assets"

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [59]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR', asset_min_buy_amount='MIN_BUY_AMOUNT'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&columns={}&columns=nav&date={}&columns=CREATION_DATE'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_min_buy_amount, asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    df_asset = df_asset[(df_asset['LAST_CLOSE_VALUE_IN_CURR'].notna()) | (df_asset['TYPE'] == 'PORTFOLIO')].reset_index() #FIXME es assets avec LAST_CLOSE_VALUE_IN_CURR = NAN bug lors de l'ajout dans un portefeuille
    return df_asset

#df_ETF = get_df_asset('2016-06-01', df_currency_rate)
#df_ETF

Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [60]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

#df_cotation = get_cotation_values('1845', '2016-06-01', '2020-09-30', df_currency_rate)
#df_cotation
    

Récupération de la liste des ratios :

In [61]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    return df_ratio

#df_ratio = get_ratio(df_currency_rate)
#df_ratio

Calcul de ratios :

In [62]:
def post_ratio(ratios, assets_id, start_date, end_date, bench=None, frequency=None):
    #start_date = datetime.strptime(start_date, "%Y-%m-%d").isoformat()
    #end_date = datetime.strptime(end_date, "%Y-%m-%d").isoformat()
    
    #payload = {'ratio':ratios, 'asset':assets_id, 'startDate':, 'endDate':None, 'benchmark':bench, 'frequency':frequency}
    payload = "{{'ratio'={},'asset'={},'start_date'={},'end_date'={}, 'bench'=null, 'frequency':null}}".format(ratios, assets_id, start_date, end_date)
    reponse = post_data('ratio/invoke', payload)
    return reponse
    
#get Rendement_annualise, Sharpe, Volatilite, Rendement de l'asset 1845 et 1991 entre '2016-06-01' et '2020-09-30'
#post_ratio([9, 12, 10, 13], [1845], '2016-06-01', '2020-09-30')

In [63]:
def get_df_ratios(assets_id, begin_date, end_date):
    json_ratios = json.loads(post_ratio([10, 12, 13], assets_id, begin_date, end_date))
    ratios = []
    for asset_id in assets_id:
        str_asset_id = str(asset_id)
        volatilite = convert_float(json_ratios[str_asset_id]['10']['value'])
        sharpe = convert_float(json_ratios[str_asset_id]['12']['value'])
        rendement = convert_float(json_ratios[str_asset_id]['13']['value'])
        ratios.append([asset_id, volatilite, rendement, sharpe])
    return pd.DataFrame(ratios, columns=['ASSET_DATABASE_ID', 'volatilite', 'rendement', 'sharpe'])

#get_df_ratios([1845, 1991], '2016-06-01', '2020-09-30')

Recupération du portfolio ID :

In [64]:
def get_portfolio_id(df_assets, label):
    portfolio_id = df_assets.loc[(df_assets['TYPE'] == 'PORTFOLIO') & (df_assets['LABEL'] == label)]['REST_OBJECT_ID'].values[0]
    return int(portfolio_id)

Récupération de la composition d’un portefeuille : 

In [65]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = pd.read_json(data)
    return df_portfolio

Get assets from portfolio :

In [66]:
def get_assets_portfolio(df_portfolio, date):
    if date not in df_portfolio['values']:
        return np.NaN
    return df_portfolio['values'][date]

MAJ de la composition d’un portefeuille :


In [67]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"]["asset"], asset["asset"]["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets=None):
    label = df_portfoliot['label'][0]#"EPITA_PTF_10"
    currency = df_portfoliot['currency'][0]#"EUR"
    type_ = df_portfoliot['type'][0]
    date = '2016-06-01'

    if assets is None:
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{}} }}'.format(label, currency, type_, date)

    else:
      assets = portfolio_json_assets(assets)
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{"{}": [{}, {{"currency": {{"currency": {{"code": "EUR"}}, "amount": {} }}  }}  ]}}}}'.format(label, currency, type_, date, assets, 300)
    #res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{ "{}": [{}] }} }}'.format(label, currency, type_, date, assets)

    return json.loads(res)

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset)
    return assets

[A CHECK le JSON de put_portfolio]

Ecriture dans le portefeuille

In [68]:
def put_portfolio(portfolio_id, df_portfolio, assets):
    content = seralize_portfolio_content(df_portfolio, assets)
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

Reset asset in portfolio :

In [69]:
def reset_assets_portfolio(portfolio_id, df_portfolio):
    content = seralize_portfolio_content(df_portfolio)
    content['values'] = {}
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

Contraites :

50% d'actions  (en montant pas en quantité)

Le portefeuille doit être exactement composé d’un minimum de 15 actifs, et d’un maximum de 40 actifs.

Attention aux devises :

Vérifier bien la devise dans laquelle est exprimée l’actif.

Avez-vous utilisé le bon taux de conversion pour la devise ?

Avez-vous utilisé le taux dans le bon sens ?

Attention aux quotités:

Votre portefeuille comporte-il bien un nombre entier d’actif ? Ou avez-vous acheté 0.0001 action Total ?

L’actif est-il achetable par unité, dizaine, centaines ? le prix donné est-il un prix par unité, dizaine, centaine ?

Le prix de l’actif est-il exprimé en unité ou en centimes ?

Si vous avez arrondis à postériori les quotités, respectez-vous bien toujours les normes fixées pour le portefeuille ?

Merge des cotations de l'asset (asset_id) le 1er jour avec les cotations du dernier jour

Que faisons-nous si la date de début ou de fin de l'asset n'est pas disponible ? on prend un jour avant ? un jour après ?

**SELECTION DES ACTIFS POUR LE PORTFOLIO :**

Génération de la voile de bateau pour 10 000 portfolios composés de tous les actifs possibles et des quantités aléatoires

In [70]:
import matplotlib.pyplot as plt
import random
#FIXME il faudrait imposer les contraintes du sujet aux portfolios générés (ex: nav 1 et 15%, stock > 50%, ...)
def get_Markowitz_Bullet(begin_date, end_date, best_assets=None, assets=None, iterations=10000):
    if assets is None:
        assets = get_df_asset(begin_date, df_currency_rate)
        assets = assets[assets['TYPE'] != 'PORTFOLIO']
    assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]
    #assets_id = random.choices(assets_id, k=5)
    
    #dataframe de tous les ratio 12 Sharpe 13 Rendement 10 Volatile pour chaque assets
    df_ratios = get_df_ratios(assets_id, begin_date, end_date)
    assets = assets.merge(df_ratios, on='ASSET_DATABASE_ID')
    
    portfolio_returns = []
    portfolio_volatilities = []
    portfolio_sharpes = []
    
    returns = assets['rendement'].values
    volatilities = assets['volatilite'].values
    sharpes = assets['sharpe'].values
    
    for i in range(iterations):
        weights = np.random.dirichlet(np.ones(assets.shape[0]), size=1)
        weights = weights[0]

        total_returns = np.dot(returns, weights)
        total_volatilities = np.dot(volatilities, weights)
        total_sharpes = np.dot(sharpes, weights)
        
        #rates_returns = (1 + returns)**(365/1582) - 1

        #portfolio_sharpes.append((np.sum(rates_returns) - 0.05)/np.sum(total_volatilities))
        #print((np.sum(taux_returns) - 0.05)/np.sum(total_volatilities))
        #print(-(taux_returns - 0.05)/np.dot(volatilities, weights))
        
        portfolio_returns.append(total_returns)
        portfolio_volatilities.append(total_volatilities)
        portfolio_sharpes.append(total_sharpes)
        
    portfolio_returns = np.array(portfolio_returns)
    portfolio_volatilities = np.array(portfolio_volatilities)
    portfolio_sharpes = np.array(portfolio_sharpes)
    
    plt.figure(figsize=(18,15))
    plt.scatter(portfolio_volatilities, portfolio_returns, c = portfolio_returns/portfolio_volatilities)
                #c=(portfolio_sharpes)) #FIXME
    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.colorbar(label='Sharpe ratio')
    plt.title('Markowitz Bullet (voile de bateau)')
    
    if best_assets is not None:
        returns = best_assets['rendement'].values
        volatilities = best_assets['volatilite'].values
        sharpes = best_assets['sharpe'].values
        best_weights = best_assets['weights'].values

        best_portfolio_returns = np.dot(returns, best_weights)
        best_portfolio_volatilies = np.dot(volatilities, best_weights)
        best_sharpe = best_portfolio_returns / best_portfolio_volatilies
        plt.scatter(best_portfolio_volatilies, best_portfolio_returns, c='red', marker='+', s=[500])

        print('Best portfolio have a sharpe ratio = {}'.format(best_sharpe))

#get_Markowitz_Bullet('2016-06-01', '2020-09-30')

In [71]:
def portfolio_stats(assets, weights):
    returns = assets['rendement'].values
    volatilities = assets['volatilite'].values
    #sharpes = assets['sharpe'].values

    total_returns = np.dot(returns, weights) #FIXME
    total_volatilities = np.dot(volatilities, weights) #FIXME

    #rates_returns = (1 + returns)**(365/1582) - 1
    #portfolio_sharpes.append((np.sum(rates_returns) - 0.05)/np.sum(total_volatilities))

    total_sharpes = total_returns / total_volatilities #FIXME
    
    return {'return': total_returns, 'volatility': total_volatilities, 'sharpe': total_sharpes}

Minimization Functions

In [72]:
def minimize_neg_sharpe(weights, assets):#on veut maximiser sharpe
    return -portfolio_stats(assets, weights)['sharpe'] 

The Optimization Function

In [96]:
"""def nav_max_constraint(weights): #FIXME ON doit check > 1 %
     assets = assets_global
     index_max = np.argmax(weights)
     asset_nav_max = assets['LAST_CLOSE_VALUE_IN_CURR'].values[index_max] #FIXME change close to nav
     weight_max = max(weights)
     a = weight_max * asset_nav_max - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.095 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_max
     res_nav_max = a
     return a

def nav_min_constraint(weights):
     assets = assets_global
     index_min = np.argmin(weights)
     asset_nav_min = assets['LAST_CLOSE_VALUE_IN_CURR'].values[index_min] #FIXME change close to nav
     weight_min = min(weights)
     a = weight_min * asset_nav_min - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.015 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_min
     res_nav_min = a
     return a"""

def nav_max_constraint(weights): #FIXME ON doit check > 1 %
     assets = assets_global
     nb_weights = len(weights)

     max_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if max_nav is None:
         max_nav = nav
       else:
         max_nav = max(max_nav, nav)
     res = max_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.095 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_max
     res_nav_max = res
     return res

def nav_min_constraint(weights):
     assets = assets_global

     nb_weights = len(weights)

     min_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if min_nav is None:
         min_nav = nav
       else:
         min_nav = min(min_nav, nav)
     res = min_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.015 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_min
     res_nav_min = res
     return res

def stock_constraint(weights):
     assets = assets_global

     stock_assets_id = [asset_id for asset_id in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_id]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_id] #FIXME close to nav

     res = np.dot(stock_weights, stock_nav) - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.6 #FIXME augmenter au-dessus de 0.5 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_assets_number
     res_assets_number = res
     return res

def optimize_sharpe(assets):
    nb_assets = assets.shape[0]

    global assets_global
    assets_global = assets
    
    constraints = ({'type': 'eq', 'fun': nav_min_constraint},
                   {'type': 'eq', 'fun': nav_max_constraint},
                   {'type': 'eq', 'fun': stock_constraint})
    bounds = tuple((1, None) for x in range(nb_assets))
    initializer = nb_assets * [1/ nb_assets]
    
    best_sharpe = optimize.minimize(minimize_neg_sharpe, initializer, args = (assets), method='SLSQP', bounds=bounds, constraints=constraints)
    return best_sharpe.x

Check constraints functions

In [94]:
def check_nav_max_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    max_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if max_nav is None:
        max_nav = nav
      else:
        max_nav = max(max_nav, nav)
    return max_nav <= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.10
     
def check_nav_min_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    min_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if min_nav is None:
        min_nav = nav
      else:
        min_nav = min(min_nav, nav)
    return min_nav >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.01 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

def check_nav_constraint(assets):
     check_max = check_nav_max_constraint(assets)
     check_min = check_nav_min_constraint(assets)

     print('check_max = {}\ncheck_min = {}'.format(check_max, check_min))
     return check_max and check_min

def check_stock_constraint(assets):
     weights = assets['weights'].values

     stock_assets_index = [asset_index for asset_index in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_index]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_index] #FIXME close to nav

     return np.dot(stock_weights, stock_nav) >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.5

def check_quantity_constraint(assets):
    for _, row in assets.iterrows():
      if ((row['MIN_BUY_AMOUNT'] is not np.NaN) and (row['weights'] < row['MIN_BUY_AMOUNT'])):
          return False
    return True

In [75]:
#get_Markowitz_Bullet('2016-06-01', '2020-09-30', best_assets=best_assets, iterations=100000)

Select best assets function :

In [101]:
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats

def select_best_assets(begin_date, end_date, df_currency_rate):
    assets = get_df_asset(begin_date, df_currency_rate)
    #remove portfolios from assets
    assets = assets[assets['TYPE'] != 'PORTFOLIO']
    assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]
    
    #dataframe de tous les ratio 12 Sharpe 13 Rendement 10 Volatile pour chaque assets
    df_ratios = get_df_ratios(assets_id, begin_date, end_date)
    assets = assets.merge(df_ratios, on='ASSET_DATABASE_ID')

    #select assets with best sharpe : #FIXME Kmeans pour la corrélation
    best_assets = assets.sort_values(by=['sharpe'], ascending=False)[:20].reset_index().drop(columns=['level_0', 'index'])#FIXME pas 20
    best_assets = best_assets[best_assets['MIN_BUY_AMOUNT'].isna()]

    global res_nav_max
    res_nav_max = 0
    global res_nav_min
    res_nav_min = 0
    global res_assets_number
    res_assets_number = 0

    weights = optimize_sharpe(best_assets)


    print(np.sort(weights))
    print('nav score max = {}'.format(res_nav_max))
    print('nav score min = {}'.format(res_nav_min))
    print('assets stock score = {}'.format(res_assets_number))

    best_assets['weights'] = (weights * 100).astype(int)
    
    print('check nav constraint : {}'.format(check_nav_constraint(best_assets)))
    print('check stock constraint : {}'.format(check_stock_constraint(best_assets)))
    print('check minimal buy amount constraint : {}'.format(check_quantity_constraint(best_assets)))

    return best_assets

Post best assets selected in our portfolio :

In [77]:
def post_assets_selected_in_portfolio(assets, begin_date, df_currency_rate):
    assets_id_quantity = [{"asset": {"asset": asset['ASSET_DATABASE_ID'], "quantity": asset['weights']}} for _, asset in assets.iterrows()]
    df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)
    
    portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
    df_portfolio = get_portfolio(portfolio_id)
    
    reset_assets_portfolio(portfolio_id, df_portfolio)#on reset le contenu du portfolio (plus safe)
    
    df_portfolio = get_portfolio(portfolio_id)
    put_portfolio(portfolio_id, df_portfolio, assets_id_quantity)

Comparaison du ratio de sharpe entre notre portefeuille et le portefeuille de référence : 

In [78]:
def compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date):    
    ratios = json.loads(post_ratio([12], [our_portfolio_id, ref_portfolio_id], begin_date, end_date))
    print('sharpe ratio in our portfolio : {}\nsharpe ratio in ref portfolio : {}'
          .format(ratios[str(our_portfolio_id)]['12']['value'], ratios[str(ref_portfolio_id)]['12']['value']))

Main function :

In [102]:
def main():
  begin_date = '2016-06-01'
  end_date = '2020-09-30'
  #df_currency_rate = getCurrencyRate()#FIXME

  assets_selected = select_best_assets(begin_date, end_date, df_currency_rate)

  post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)

  df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)

  our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
  our_portfolio = get_portfolio(our_portfolio_id)

  reset_assets_portfolio(our_portfolio_id, our_portfolio)#FIXME ATTENTIN A SUPRIMMER AVANT DE POST POUR AVOIR LES ACTIFS DANS LE PORTFOLIO

  our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
  ref_portfolio_id = get_portfolio_id(df_assets, 'REF')

  compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)

main()

[1.21364362e+00 2.47505838e+00 2.76034230e+01 3.90935854e+01
 4.45124852e+01 8.52595920e+01 9.61960720e+01 4.95887358e+02
 6.37926335e+02 7.68860119e+02 9.74790029e+02 1.00283156e+03
 1.05982006e+03 1.08907970e+03 2.26043193e+03 5.38644624e+03
 7.05976951e+03 7.72439330e+03 1.47863717e+05]
nav score max = 0.02983515601954423
nav score min = 9.522336767986417e-07
assets stock score = 5.739170592278242e-05
check_max = True
check_min = True
check nav constraint : True
check stock constraint : True
check minimal buy amount constraint : True
sharpe ratio in our portfolio : 2,737628113474
sharpe ratio in ref portfolio : -0,337925081462


In [41]:
begin_date = '2016-06-01'
end_date = '2020-09-30'
assets = get_df_asset(begin_date, df_currency_rate)
assets_id = [asset_id for asset_id in assets['ASSET_DATABASE_ID'].values]

nav_values = []
for asset_id in assets_id:
  nav = get_cotation_values(asset_id, begin_date, begin_date, df_currency_rate)['nav'].values[0]
  nav_values.append(nav)

nav_values

KeyError: ignored

In [40]:
get_cotation_values(1845, begin_date, '2020-09-30', df_currency_rate)

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2016-06-01,36.851000000000,36.851000000000,143199.0,37.125,36.851000000000,-10.0,36.675,36.799,1845,0.037000000000,36.723,36.851000000000,0.001005052426,36.745
1,2016-06-02,36.900000000000,36.900000000000,147480.0,37.145,36.900000000000,-10.0,36.594,36.999,1845,0.049000000000,36.946,36.900000000000,0.001329678978,36.786
2,2016-06-03,36.052000000000,36.052000000000,190934.0,37.200,36.052000000000,-10.0,35.800,36.277,1845,-0.848000000000,36.219,36.052000000000,-0.022981029810,36.900
3,2016-06-06,36.540000000000,36.540000000000,82279.0,36.775,36.540000000000,-10.0,36.082,36.587,1845,0.488000000000,36.534,36.540000000000,0.013536003550,36.190
4,2016-06-07,37.246000000000,37.246000000000,119105.0,37.347,37.246000000000,-10.0,36.470,37.172,1845,0.706000000000,37.150,37.246000000000,0.019321291735,36.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2020-09-24,27.091248954261,27.091248954261,NaN,NaN,27.091248954261,-10.0,NaN,NaN,1845,-0.550081510430,NaN,27.091248954261,-0.019900688613,NaN
1117,2020-09-25,26.877384954417,26.877384954417,NaN,NaN,26.877384954417,-10.0,NaN,NaN,1845,-0.213863999844,NaN,26.877384954417,-0.007894209684,NaN
1118,2020-09-28,26.634573242213,26.634573242213,NaN,NaN,26.634573242213,-10.0,NaN,NaN,1845,-0.242811712203,NaN,26.634573242213,-0.009034052703,NaN
1119,2020-09-29,26.368478372669,26.368478372669,NaN,NaN,26.368478372669,-10.0,NaN,NaN,1845,-0.266094869544,NaN,26.368478372669,-0.009990581307,NaN


In [37]:
data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets

,SECTOR,LIQUIDITY_ALGO,MARKET_PLACE_CITY,MARKET_PLACE_TYPE,SUP_SECTOR,VL_DELAY,CODE_CIC,INDEXED_RATE,COUNTRY_CIC,IS_EURONEXT,PORT_CONTENT_COUNT,LAST_CLOSE_VALUE_IN_CURR,FLOATING_VALUE,CREATION_DATE,SUB_TYPE,IS_STRUCTURED_PRODUCT,IS_ACCESSIBILITY_PUBLIC,IS_FUND_SAVE_SENS,PORT_LINE_COUNT,ASSET_DATABASE_ID,ASSET PRICE MODE,GICS_SUB_INDUSTRY,MARKET_PLACE_TELEKURS,ISIN_CODE,IS_EUROLAND,MARKET_PLACE,GRAPH_LAST_YEAR,PAY_ACCOUNT_ON_DEFAULT,GRAPH_PAST_WEEK,GICS_SECTOR,MARKET_PLACE_CURRENCY,GRAPH_LAST_6_MONTH,asset_custom_classification_bloomberg,LAST_QUOTE_SOURCE,MARKET_PLACE_MIC,GICS_INDUSTRY,IS_DEPOSITARY_QUOTES,CITY_OR_ACRONYM_OR_NAME,DAILY_PRICE_COUNT,MODIFICATION_DATE,SUB_CATEGORY_CIC,FINAL_UNDERLYING,IS_COMPOSITE,REST_UNIQUE_NUMBER,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,MARKET_PLACE_COUNTRY,BLOOMBERG_CODE,GICS_INDUSTRY_GROUP,quote_value_type,CURRENCY,LABEL,INDUSTRY,LAST_CLOSE_VALUE,IS_FUND_FED,LAST_QUOTES_UPDATE_DATE,LAST_GROSS_VALUE,BLOOMBERG_ID,MARKET_CAP_CATEGORY,ASSET_STATUS,FINAL_UNDERLYING_TYPE,MARKET_PLACE_OR_ASSET_COUNTRY,CATEGORY_CIC,REST_OBJECT_ID,GRAPH_PAST_YEAR,GRAPH_PERIOD,LAST_QUOTE_DATE,SUB_SECTOR,STOCK_TYPE,GLOBAL_TYPE,IS_PUBLICLY_TRADED,SEDOL_CODE,MARKET_PLACE_DESC,ASSET_OR_MARKET_PLACE_COUNTRY,QUOTE_TYPE,EMIR_CLEARING_ID_TYPE,TYPE,COUNTRY,asset_attr_init,FIRST_QUOTE_DATE,IS_PORTFOLIO,COUNTRY_ISO_3,asset_custom_classification_Tab. de bord GSM,ASSET_MARKET_SUB_GROUP,ELIGIBLE_PEA_PME,ELIGIBLE_PEA,IS_REAL_ESTATE_FUND,asset_fund_info_isin_code,asset_fund_info_objective_long,asset_fund_info_cut_off,asset_fund_info_administrator,asset_fund_info_manager_stated_perf_fee,asset_fund_info_authorized_for_retail_investors,BOND_CREDIT_RISK_COMPUTATION_MODE,FUND_TYPE,asset_fund_info_creation_date,asset_fund_info_vl_date,asset_fund_info_is_feeded,asset_fund_info_is_euro_ucits,asset_fund_info_asset_class_focus,asset_fund_info_maximum_front_end_fee,TRANSPARENCY_LIST,asset_fund_info_strategy,BOND_WAL_COMPUTATION_MODE,asset_fund_info_manager_stated_fee,asset_fund_info_valorisation_frequency,BOND_RISK_COMPUTATION_MODE,BOND_WAM_COMPUTATION_MODE,asset_fund_info_auditor,asset_fund_info_fund_leverage,asset_fund_info_maximum_back_end_fee,BOND_YTM_COMPUTATION_MODE,asset_fund_info_fund_type,asset_fund_info_minimum_investment_shares,asset_fund_info_bloomberg_ticker,asset_fund_info_company,asset_fund_info_depositary,MIN_BUY_AMOUNT,asset_fund_info_subscription_committee,asset_fund_info_geographical_focus,asset_fund_info_day_cut_off,asset_fund_info_minimum_subscription,MIN_BUY_INITIAL_AMOUNT,asset_fund_info_vl_known,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_date,asset_fund_info_decimalisation,asset_fund_info_amf_classification,asset_fund_info_abide_with_the_four_criterias,asset_fund_info_reuters_code,REUTERS_CODE,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,asset_fund_info_rate_entry,asset_fund_info_rate_output,asset_fund_info_management_style,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,TELEKURS_QUOTE_PLACE_CODE,QUOTE_PLACE,IS_SRD,PORT_FEED_CODE,PORT_RATING_MOODYS_SHORT_TERM,PORT_NET_EX_CASH_EQUIVALENT,PORT_NET_EX_CASH,LOCK_MODE,EAI_PORT_AVAILABLE_CASH,PORT_CLEAN_MODE,NB_COMPTE_ON_PORTFOLIO,PORT_CASH_EQUIVALENTS,EAI_PORTFOLIO_FEED_NAME,PORT_RATING_SP_SHORT_TERM,PORT_BASE_NAV,PORT_CURRENCY_HEDGE,PORT_IS_HISTORICAL,PORT_FIRST_TRANSACTION,PORT_RATING_MOODYS,PORT_RATING_SP,PORT_NEED_RECONCILIATION,PRICER_MODE,PORT_TYPE,REAL_TIME_PORT_CASH,PORT_FEED_TYPE,PORT_STATUS,PORT_VL,PORT_RATING_FITCH,EAI_FRONTPORT_CASH_BY_CRCY,PORT_CASH_AVAILABLE,PORT_RATING_FITCH_SHORT_TERM,EAI_BACKPORT_CASH_BY_CRCY,PORT_CASH,VL_DE